In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Hide warnings
import warnings
warnings.filterwarnings('ignore')

In [44]:
import tensorflow as tf

# TensorBoard setup

#### Tip: Setup TensorBoard if you want to monitor and analyze computational graphs etc

In [45]:
from datetime import datetime
import os
import pathlib

t = datetime.utcnow().strftime("%Y%m%d%H%M%S") 
log_dir = "tf_logs"
logd = "{}/r{}/".format(log_dir, t)

# Make directory if it doesn't exist

from pathlib import Path
home = str(Path.home())

logdir = os.path.join(os.sep,home,logd)

if not os.path.exists(logdir):
    os.makedirs(logdir)

In [46]:
logdir

'C:\\Users\\shubh\\tf_logs/r20211103124219/'

# Look at the computational graph with @tf.function

#### @tf.function is a very useful module taht can be used to convert simple python functons into a highly optimized computational graph that can be run on any runtime environment. When we build a model and then train it TensorFlow we can compile the model and optimize the executions

In [47]:
@tf.function
def func2(a,b):
    z = tf.multiply(a,b,name='multiplying')
    y1 = tf.constant(3,name='declaring_y1')
    y2 = tf.constant(4)
    w1 = tf.add(z,y1,name='adding_y1_to_z')
    w2 = tf.add(z,y2,name='adding_y2_to_z')
    
    return(w1+w2)

In [48]:
func2(10,20)

<tf.Tensor: shape=(), dtype=int32, numpy=407>

In [49]:
@tf.function
def func(a,b):
    with tf.name_scope('first'):
        z = tf.multiply(a,b,name='multiplying')
    with tf.name_scope('second'):
        y1 = tf.constant(3,name='declaring_y1')
        y2 = tf.constant(4)
        w1 = tf.add(z,y1,name='adding_y1_to_z')
        w2 = tf.add(z,y2,name='adding_y2_to_z')
    
    return(w1+w2)

In [50]:
# setup a writer to save graph information and TensorFLow logs
# to be displayed with Tensorboard

writer = tf.summary.create_file_writer(logdir)
tf.summary.trace_on()

In [51]:
a = tf.constant(3)
b = tf.constant(4)
func(a,b)
with writer.as_default():
    tf.summary.trace_export(
        name="func",
        step=0,
        profiler_outdir=logdir
    )

In [52]:
logdir

'C:\\Users\\shubh\\tf_logs/r20211103124219/'

In [53]:
# run tensorboard in the shell
!tensorboard --logdir $logdir

^C


## tf.function and conditional statements

#### It is difficult to use conditions in graphs but we could implement that easily using @tf.function decorator

In [54]:
@tf.function
def g(x):
    y = tf.reduce_sum(x)
    if y>0:
        return y
    return tf.abs(y)

In [55]:
print(tf.autograph.to_code(g.python_function))

def tf__g(x):
    with ag__.FunctionScope('g', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        y = ag__.converted_call(ag__.ld(tf).reduce_sum, (ag__.ld(x),), None, fscope)

        def get_state():
            return (do_return, retval_)

        def set_state(vars_):
            nonlocal do_return, retval_
            (do_return, retval_) = vars_

        def if_body():
            nonlocal do_return, retval_
            try:
                do_return = True
                retval_ = ag__.ld(y)
            except:
                do_return = False
                raise

        def else_body():
            nonlocal do_return, retval_
            try:
                do_return = True
                retval_ = ag__.converted_call(ag__.ld(tf).abs, (ag__.ld(y),), None, fscope)
            except:
                do_return 

# Calculate gradients

#### Gradient evaluation is very importnat machine learning because it is based on function optimization. You can use tf.GradientTape() method to record the gradient of an arbitrary function

In [57]:
w = tf.Variable(3.0)

# Gradient scope for the function e^w + 2*w + w^2 
with tf.GradientTape() as tape:
    expression = tf.exp(w) + 2 * w +  w * w

grad = tape.gradient(expression, w)
print(f'The gradient of exp at {w.numpy()} is {grad.numpy()}')

The gradient of exp at 3.0 is 28.08553695678711


## Gradient of the Sigmoid function

In [58]:
def sigmoid(x):
    return 1/(1 + tf.exp(-x))

In [59]:
#define a varaible
x = tf.Variable(0.)

#record the gradient
with tf.GradientTape() as tape:
    sig = sigmoid(x)
    
res = tape.gradient(sig, x).numpy()
print('The gradient of the sigmoid function at 0.0 is ', res)

The gradient of the sigmoid function at 0.0 is  0.25
